### Financial Analysis - Multi-Agent Pattern

This sample code demonstrates the use of a multi-agent pattern for analysing complex financial data for reporting. By using agents specialising in analysing areas such as profitability, capitalisation & asset quality, a summary containing the financial analysis is generated. The agents make use of the Assistants API in Azure OpenAI.

The multi-agent example published by Microsoft has been used as a reference for this notebook - https://github.com/Azure-Samples/azureai-samples/blob/main/scenarios/Assistants/multi-agent/multi-agent.ipynb


In [ ]:
# Uncomment lines below to install dependencies; re-comment after installation.
# %pip install -r requirements.txt
# %pip install --upgrade openai

In [ ]:
# Set-up
import os
import time
from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types.beta import Thread
from openai.types.beta import Assistant

load_dotenv()

assistant_client = AzureOpenAI(
    api_key=os.getenv("GPT4_AZURE_OPENAI_KEY"),
    api_version=os.getenv("GPT4_AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv(
        "GPT4_AZURE_OPENAI_ENDPOINT"
    ),
)

assistant_deployment_name = os.getenv(
    "GPT4_DEPLOYMENT_NAME"
)

In [ ]:
# Prompt - Profitability Assistant
name_profitability_analyser = "profitability_assistant"
instructions_profitability_analyser = """""" # Add prompt for proftability analysis here

In [ ]:
# Prompt - Capitalisation Assistant
name_capitalisation_analyser = "capitalisation_assistant"
instructions_capitalisation_analyser = """""" # Add prompt for capitalisation analysis here

In [ ]:
# Prompt - Asset Quality Assistant
name_assetquality_analyser = "assetquality_assistant"
instructions_assetquality_analyser = """""" # Add prompt for asset quality analysis here

In [ ]:
# Prompt - Summariser Assistant
name_summariser = "summariser_assistant"
instructions_summariser = """""" # Add prompt for generating the summary here

In [ ]:
# Create Assistants
profitability_assistant = assistant_client.beta.assistants.create(
    name=name_profitability_analyser, instructions=instructions_profitability_analyser, model=assistant_deployment_name, tools=[{"type":"code_interpreter"},{"type":"file_search"}], temperature=0.01
)

capitalisation_assistant = assistant_client.beta.assistants.create(
    name=name_capitalisation_analyser, instructions=instructions_capitalisation_analyser, model=assistant_deployment_name, tools=[{"type":"code_interpreter"},{"type":"file_search"}], temperature=0.01
)

assetquality_assistant = assistant_client.beta.assistants.create(
    name=name_assetquality_analyser, instructions=instructions_assetquality_analyser, model=assistant_deployment_name, tools=[{"type":"code_interpreter"},{"type":"file_search"}], temperature=0.01
)

summariser_assistant = assistant_client.beta.assistants.create(
    name=name_summariser, instructions=instructions_summariser, model=assistant_deployment_name, temperature=0.01
)

In [ ]:
# Create Vector Store
vector_store = assistant_client.beta.vector_stores.create(name="Financial Statements")

file_paths = [""] # Add path for financial statements here
file_streams = [open(path, "rb") for path in file_paths]

file_batch = assistant_client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
    )

In [ ]:
# Attach Vector Store to Assistants
profitability_assistant = assistant_client.beta.assistants.update(
    assistant_id=profitability_assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
      )

capitalisation_assistant = assistant_client.beta.assistants.update(
    assistant_id=capitalisation_assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
      )

assetquality_assistant = assistant_client.beta.assistants.update(
    assistant_id=assetquality_assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
      )

In [ ]:
# Create Files for Assistants
file_profitability = assistant_client.files.create(
  file=open("", "rb"), # Add path for file with profitability metrics here
  purpose='assistants'
)

file_capitalisation = assistant_client.files.create(
  file=open("", "rb"), # Add path for file with capitalisation metrics here
  purpose='assistants'
)

file_assetquality = assistant_client.files.create(
  file=open("", "rb"), # Add path for file with asset quality metrics here
  purpose='assistants'
)

In [ ]:
# Attach Files to Assistants
profitability_assistant = assistant_client.beta.assistants.update(
    assistant_id=profitability_assistant.id,
    tool_resources={"code_interpreter": {"file_ids": [file_profitability.id]}}
      )

capitalisation_assistant = assistant_client.beta.assistants.update(
    assistant_id=capitalisation_assistant.id,
    tool_resources={"code_interpreter": {"file_ids": [file_capitalisation.id]}}
      )

assetquality_assistant = assistant_client.beta.assistants.update(
    assistant_id=assetquality_assistant.id,
    tool_resources={"code_interpreter": {"file_ids": [file_assetquality.id]}}
      )

In [ ]:
# Create Thread
thread = assistant_client.beta.threads.create()

message = assistant_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Analyse."
)

In [ ]:
# Function for Running Assistants
def run_assistant(assistant):
  run = assistant_client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
  )

  while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = assistant_client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )

  if run.status == 'completed':
    messages = assistant_client.beta.threads.messages.list(
      thread_id=thread.id, 
      run_id=run.id
    )
  elif run.status == 'requires_action':
    pass
  else:
    print(run.status)

  return messages.data

In [ ]:
# Function for Printing Analysis
def print_analysis(analysis):
    for msg in reversed(analysis):
        for content_item in msg.content:
            print(content_item.text.value)

In [ ]:
# Run Profitability Assistant
analysis = run_assistant(profitability_assistant)
print_analysis(analysis)

In [ ]:
# Run Capitalisation Assistant
analysis = run_assistant(capitalisation_assistant)
print_analysis(analysis)

In [ ]:
# Run Asset Quality Assistant
analysis = run_assistant(assetquality_assistant)
print_analysis(analysis)

In [ ]:
# Run Summariser Assistant
analysis = run_assistant(summariser_assistant)
print_analysis(analysis)

In [ ]:
# Clean-up
response = assistant_client.beta.assistants.delete(profitability_assistant.id)
response = assistant_client.beta.assistants.delete(capitalisation_assistant.id)
response = assistant_client.beta.assistants.delete(assetquality_assistant.id)
response = assistant_client.beta.assistants.delete(summariser_assistant.id)